### **Script base: Spazzata di voltaggio**

Questo script esegue una spazzata lineare dal voltaggio `V0` al voltaggio `V1` con  `nV` punti. Per ogni valore l'$\texttt{ADC}$ esegue una misura con i seguenti parametri di campionamento:

* frequenza di sampling `fs`
* numero di acquisizioni `npt`

I valori risultanti di $\texttt{Ch1}$ e $\texttt{Ch2}$ vengono mediati e salvati.

In [25]:
import tdwf
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt 
import numpy as np
import time

V0 = 1.0
V1 = 3.0
nV = 101
npt = 101
fs  = 1e6

Save = True

Vv = np.linspace(V0,V1,nV)

# -[Configurazione AD2]--------------------------------------------------------
#   1. Connessiene con AD2 e selezione configurazione 
ad2 = tdwf.AD2()
ad2.vdd = 5
ad2.power(True)
#   3. Configurazione generatore funzioni
wgen = tdwf.WaveGen(ad2.hdwf)
wgen.w1.func = tdwf.funcDC
#wgen.w1.offs = V0
wgen.w1.start()
#   3. Configurazione oscilloscopio
scope = tdwf.Scope(ad2.hdwf)
scope.fs=fs
scope.npt=npt
scope.ch1.avg=True
scope.ch2.avg=True
scope.ch1.rng = 50
scope.ch2.rng = 50

# -[Ciclo di misura]-----------------------------------------------------------
fig, [ax1, ax2] = plt.subplots(1, 2, figsize=(12, 6))
fig.canvas.manager.set_window_title('Spazzata voltaggio')
ax2.yaxis.tick_right()
Ch1v = np.full(nV, np.nan)
s_Ch1v = np.full(nV, np.nan)
Ch2v = np.full(nV, np.nan)
s_Ch2v = np.full(nV, np.nan)

for ii, V in enumerate(Vv):
    wgen.w1.offs = V
    time.sleep(0.1)
    scope.sample()
    Ch1v[ii] = np.mean(scope.ch1.vals)
    s_Ch1v[ii] = np.std(scope.ch1.vals)
    Ch2v[ii] = np.mean(scope.ch2.vals)
    s_Ch2v[ii] = np.std(scope.ch2.vals)

    ax1.clear()    
    ax1.errorbar(Vv, Ch1v, s_Ch1v, fmt=".", color="tab:orange", label="Ch1")
    ax1.errorbar(Vv, Ch2v, s_Ch2v, fmt=".", color="tab:blue", label="Ch2")
    ax1.grid(True)
    ax1.set_xlabel("W1 [V]", fontsize=15)
    ax1.set_ylabel("Signals [V]", fontsize=15)
    ax1.legend()
    
    ax2.clear()    
    ax2.errorbar(Ch1v, Ch2v, s_Ch2v, s_Ch1v, ".", color="tab:orange")
    ax2.grid(True)
    ax2.set_xlabel("Ch1 [V]", fontsize=15)
    ax2.set_ylabel("Ch2 [V]", fontsize=15)
    ax2.yaxis.set_label_position('right')
    plt.tight_layout()
    plt.show(block=False)
    plt.pause(0.001)

ad2.close()

R_1 = 680

I = (Ch1v - Ch2v) / R_1

s_I = np.sqrt(s_Ch1v**2 + s_Ch2v**2) / R_1      # ATTENZIONE, se calcoli così l'errore sulla corrente che attraversa il LED stai assumendo la resistenza esattamente nota, senza incertezza

fig1 = plt.figure('Plot Curca Caratteristica', figsize=(10, 6), dpi=100)

ax3 = fig1.subplots(1, 1)

ax3.errorbar(Ch2v, I+1e-6, s_I, s_Ch2v, fmt='.', label='curva caratteristica LED rosso')
ax3.set_ylabel('I[A]')
ax3.set_yscale('log')
ax3.set_xlabel('V_LED [V]')
ax3.grid(color='lightgray', ls='dashed')
ax3.set_xlim(1.62, 1.87)
ax3.legend()
if Save:
    path2 = "/home/marco/Desktop/Uni_anno3/TD/Es_06/logbook/"
    name = "characteristic_curve_LED_rosso"
    plt.savefig(path2+name+".png", dpi=300)

plt.show()

datas = np.array([Ch2v, s_Ch2v, I, s_I])

if Save:
    path1 = "/home/marco/Desktop/Uni_anno3/TD/Es_06/acquisizioni/"
    name = "characteristic_curve_LED_rosso"
    np.savetxt(path1+name+".txt", datas, delimiter=',', 
           header="#Ch2[V], #s_Ch2[V], #I[A], #s_I[A]")


Dispositivo #1 [SN:210321B5D136, hdwf=1] connesso!
Configurazione #1
Dispositivo disconnesso.
